In [0]:
pwd

'/content'

In [0]:
from google.colab import drive
drive.mount('./drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at ./drive


In [0]:
!ls './drive/My Drive/Thesis Work'

 CambridgeMonoPhoneDf.pickle	      jawadmodel2.h5
 CambridgeWordFeaturesDf.pickle       L2monoPhoneDfTest.pickle
 CheckScrapSpeed.ipynb		      L2monoPhoneDfTrain.pickle
 ConvertToWav.ipynb		      L2WordFeaturesDfTest.pickle
'Copy of l2arctic_release_v2.0.zip'   L2WordFeaturesDfTrain.pickle
 Implementation1		      ModelFitting.ipynb
 Implementation2		      MostCommon3000Oxford.txt
 Implementation3		      UKpronunciations
 Implementation4		      USpronunciations


Bism  
# Imports

In [1]:
import pandas as pd
import numpy as np
import os
import IPython.display as ipd
import librosa
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('ggplot')
from tqdm import tqdm_notebook


#SKLEARN
from sklearn.preprocessing import LabelBinarizer
import sklearn.metrics as sklm

#KERAS
from keras import models
from keras.models import Model, load_model
from keras import callbacks
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, AveragePooling2D
from keras.optimizers import SGD, Adam

Using TensorFlow backend.


# Implementation 1  (Word Level; Single Phone Mistake)
## Using Cambridge as Training set
## L2 as val+test set (Some utterances extracted)

### Training

#### X_TRAIN

In [0]:
train_featureDf = pd.read_pickle("./drive/My Drive/Thesis Work/wordFeaturesCambridgeDf.pickle")
train_featureDf['deltaC'] = train_featureDf.MFCC.apply(librosa.feature.delta)
train_featureDf['doubleDeltaC'] = train_featureDf.MFCC.apply(librosa.feature.delta, order=2)

In [0]:
train_featureDf.head()

,MFCC,deltaC,doubleDeltaC
a,"[[-257.79550815668307, -183.138793075057, -158...","[[9.487483812627765, 9.487483812627765, 9.4874...","[[-6.377124743043098, -6.377124743043098, -6.3..."
abandon,"[[-346.25638641643053, -241.0982570548954, -19...","[[-11.18222976760626, -11.18222976760626, -11....","[[-22.477725266663054, -22.477725266663054, -2..."
abandoned,"[[-248.55651601008637, -192.06546028812613, -1...","[[-17.25558163720832, -17.25558163720832, -17....","[[-19.623024854203184, -19.623024854203184, -1..."
ability,"[[-190.396950202404, -171.73126437760413, -190...","[[-23.270463171473146, -23.270463171473146, -2...","[[-9.370769637005102, -9.370769637005102, -9.3..."
able,"[[-296.0994640205396, -236.937433826098, -170....","[[11.507995202897401, 11.507995202897401, 11.5...","[[-9.297602021293882, -9.297602021293882, -9.2..."


In [0]:
x_train = np.array((train_featureDf.MFCC,train_featureDf.deltaC,train_featureDf.doubleDeltaC))
x_train.shape

(3, 3422, 13, 118)

In [0]:
x_train = np.transpose(x_train,[1,2,3,0])
x_train.shape

(3422, 13, 118, 3)

In [0]:
np.save('./drive/My Drive/Thesis Work/Implementation1/x_train.npy',x_train) #Feature vector

In [0]:
x_train.shape

(3422, 13, 118, 3)

_______

#### Y_TRAIN

In [0]:
train_words = np.array(train_featureDf.index)
np.save('./drive/My Drive/Thesis Work/Implementation1/train_words.npy', train_words)

In [0]:
encoder = LabelBinarizer()
y_train = encoder.fit_transform(train_words)
y_train.shape

(3422, 3422)

In [0]:
np.save('./drive/My Drive/Thesis Work/Implementation1/y_train.npy',y_train) #save y_train

#### MODEL

In [0]:
model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(32, (3, 3),padding='same', activation='relu', input_shape=(13, 118, 3)))
model.add(Conv2D(32, (3, 3),padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3),padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(3422, activation='softmax'))

callbacklist=[callbacks.History(),callbacks.ModelCheckpoint('./jawadmodel.h5', monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=20)]
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=3422, epochs=500, callbacks=callbacklist)

Epoch 1/500
3422/3422 [==============================] - 5s 2ms/step - loss: 12.4403 - acc: 2.9223e-04
Epoch 2/500
3422/3422 [==============================] - 1s 398us/step - loss: 10.9398 - acc: 5.8445e-04
Epoch 3/500
3422/3422 [==============================] - 1s 387us/step - loss: 9.1658 - acc: 2.9223e-04
Epoch 4/500
3422/3422 [==============================] - 1s 387us/step - loss: 8.4022 - acc: 2.9223e-04
Epoch 5/500
3422/3422 [==============================] - 1s 389us/step - loss: 8.1923 - acc: 5.8445e-04
Epoch 6/500
3422/3422 [==============================] - 1s 389us/step - loss: 8.1436 - acc: 5.8445e-04
Epoch 7/500
3422/3422 [==============================] - 1s 388us/step - loss: 8.1318 - acc: 5.8445e-04
Epoch 8/500
3422/3422 [==============================] - 1s 389us/step - loss: 8.1263 - acc: 0.0012
Epoch 9/500
3422/3422 [==============================] - 1s 387us/step - loss: 8.1214 - acc: 8.7668e-04
Epoch 10/500
3422/3422 [==============================] - 1s 383us/s

KeyboardInterrupt: ignored

In [0]:
!cp ./jawadmodel.h5 "./drive/My Drive/Thesis Work/Implementation1/model.h5"

In [0]:
!ls "./drive/My Drive/Thesis Work/Implementation1/"

model.h5  train_words.npy  x_train.npy	y_train.npy


##### AutoEncoder approach

In [0]:
x_train = np.load('./drive/My Drive/Thesis Work/Implementation1/x_train.npy')

In [0]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 13, 118, 3)        0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 12, 116, 32)       608       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 6, 58, 32)         0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 6, 58, 32)         15392     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 3, 29, 32)         0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 3, 29, 32)         15392     
_________________________________________________________________
up_sampling2d_15 (UpSampling (None, 6, 58, 32)         0         
__________

In [0]:
input_img = Input(shape=(13, 118, 3))  # adapt this if using `channels_first` image data format

x = Conv2D(32, (2, 3), activation='relu', padding='valid')(input_img)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='valid')(x)
x = Conv2D(32, (3, 5), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
encoded = MaxPooling2D((2, 2), padding='valid')(x)

# at this point the representation is (3, 28, 32)

x = Conv2D(32, (3, 5), activation='relu', padding='same')(encoded)
x = BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 5), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(3, (3, 5), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
decoded = Conv2DTranspose(3, (2,3), activation='relu', padding='valid')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer= keras.optimizers.Adam(), loss='mean_squared_error')
callbacklist=[callbacks.History(),callbacks.ModelCheckpoint('./jawadmodel.h5', monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=5)]
autoencoder.fit(x_train, x_train, batch_size=3422, epochs=50, callbacks=callbacklist)

Epoch 1/50
3422/3422 [==============================] - 5s 1ms/step - loss: 5437.7695
Epoch 2/50
3422/3422 [==============================] - 2s 488us/step - loss: 5434.9141
Epoch 3/50
3422/3422 [==============================] - 2s 483us/step - loss: 5432.8257
Epoch 4/50
3422/3422 [==============================] - 2s 479us/step - loss: 5431.2148
Epoch 5/50
3422/3422 [==============================] - 2s 481us/step - loss: 5430.0112

Epoch 00005: loss improved from inf to 5430.01123, saving model to ./jawadmodel.h5
Epoch 6/50
3422/3422 [==============================] - 2s 508us/step - loss: 5429.2251
Epoch 7/50
3422/3422 [==============================] - 2s 486us/step - loss: 5428.5376
Epoch 8/50
3422/3422 [==============================] - 2s 478us/step - loss: 5427.7676
Epoch 9/50
3422/3422 [==============================] - 2s 476us/step - loss: 5427.1597
Epoch 10/50
3422/3422 [==============================] - 2s 478us/step - loss: 5426.7290

Epoch 00010: loss improved from 5430

### TESTING

#### Representation Layer, L2 norm, Prediction Checks

In [0]:
!ls './drive/My Drive/Thesis Work/Implementation1'

model.h5  train_words.npy  x_train.npy	y_train.npy


loss: 0.0386 - acc: 0.9737 ==> Trained Model

In [0]:
model = load_model('./drive/My Drive/Thesis Work/Implementation1/model.h5')

In [0]:
x_train = np.load("./drive/My Drive/Thesis Work/Implementation1/x_train.npy")
y_train = np.load("./drive/My Drive/Thesis Work/Implementation1/y_train.npy")

In [0]:
train_words = np.load("./drive/My Drive/Thesis Work/Implementation1/train_words.npy") #vocabulary

In [0]:
encoder = LabelBinarizer()
encoder.fit(train_words)

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [0]:
encoder.inverse_transform(model.predict(x_train[:4])) #predicting correctly

array(['a', 'abandon', 'abandoned', 'ability'], dtype='<U16')

In [0]:
encoder.inverse_transform(y_train[:4])

array(['a', 'abandon', 'abandoned', 'ability'], dtype='<U16')

In [0]:
np.all(encoder.inverse_transform(y_train) == train_words) #decoder also working fine

True

In [0]:
train_words = list(train_words) #training classes AKA vocabulary

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 13, 118, 32)       896       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 13, 118, 32)       9248      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 6, 59, 32)         0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 6, 59, 64)         18496     
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 4, 57, 64)         36928     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 2, 28, 64)         0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 3584)              0         
__________

In [0]:
model.layers[-2] #representation

In [0]:
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(index=-2).output)

In [0]:
np.linalg.norm(intermediate_layer_model.predict(x_train[1:2]) - intermediate_layer_model.predict(x_train[1:2])) #l2 norm

0.0

#### CREATING TEST ARRAYS

In [0]:
test_featureDf = pd.read_pickle("./drive/My Drive/Thesis Work/L2WordFeaturesDfTest.pickle")
test_featureDf['deltaC'] = test_featureDf.MFCC.apply(librosa.feature.delta)
test_featureDf['doubleDeltaC'] = test_featureDf.MFCC.apply(librosa.feature.delta, order=2)

In [0]:
test_featureDf.head()

,word,MFCC,mispronouncedPhones,correctPhones,phoneErrorPercentage,deltaC,doubleDeltaC
0,god,"[[-421.61388650494547, -411.03346377162995, -4...",[],"[G, AA1, D]",0.00,"[[22.129047977480177, 22.129047977480177, 22.1...","[[9.704801201092947, 9.704801201092947, 9.7048..."
1,bless,"[[-447.436674575593, -452.4805757789892, -464....",[],"[B, L, EH1, S]",0.00,"[[27.87373473523671, 27.87373473523671, 27.873...","[[-1.82042100435619, -1.82042100435619, -1.820..."
2,em,"[[-458.1490641948307, -439.05567855356315, -40...",[sil],"[AH0, M]",33.33,"[[13.654831133953266, 13.654831133953266, 13.6...","[[-3.934583327436367, -3.934583327436367, -3.9..."
3,i,"[[-485.31037645101736, -465.941108477538, -459...",[],[AY1],0.00,"[[16.64703947610951, 16.64703947610951, 16.647...","[[-4.245814334509688, -4.245814334509688, -4.2..."
4,hope,"[[-371.1558491162834, -365.84428452738575, -37...",[],"[HH, OW1, P]",0.00,"[[6.707173018420316, 6.707173018420316, 6.7071...","[[3.105683962095327, 3.105683962095327, 3.1056..."


In [0]:
print("Test Words: ", len(test_featureDf.word))
print("Unique Test Words: ", len(set(test_featureDf.word)))

Test Words:  9140
Unique Test Words:  262


In [0]:
x_test = np.array((test_featureDf.MFCC,test_featureDf.deltaC,test_featureDf.doubleDeltaC))
x_test.shape

(3, 9140, 13, 118)

In [0]:
x_test = np.transpose(x_test,[1,2,3,0])

In [0]:
x_test.shape

(9140, 13, 118, 3)

In [0]:
test_words = np.array(test_featureDf.word)
test_phoneErrorPercentage = np.array(test_featureDf["phoneErrorPercentage"])
print(test_words.shape)
print(test_phoneErrorPercentage.shape)

(9140,)
(9140,)


In [0]:
np.save('./drive/My Drive/Thesis Work/Implementation2/x_test.npy',x_test)
np.save('./drive/My Drive/Thesis Work/Implementation2/test_phoneErrorPercentage.npy',test_phoneErrorPercentage)
np.save('./drive/My Drive/Thesis Work/Implementation2/test_words.npy',test_words)

#### LOADING ARRAYS, VALIDATION AND TESTING

In [2]:
!ls './drive/My Drive/Thesis Work/WordLevel_RL_MP_MFCC_TrainL2_TestL2/'

model.h5		       test_words.npy	 x_train.npy
test_correctPhones.npy	       train_phones.npy  y_train.npy
test_mispronouncedPhones.npy   train_words.npy
test_phoneErrorPercentage.npy  x_test.npy


In [3]:
!ls './drive/My Drive/Thesis Work/WordLevel_RL_MP_MFCC_TrainCamb_TestL2/'

model.h5		       test_words.npy	x_test.npy   y_train.npy
test_phoneErrorPercentage.npy  train_words.npy	x_train.npy


In [5]:
x_train = np.load('./drive/My Drive/Thesis Work/WordLevel_RL_MP_MFCC_TrainCamb_TestL2/x_train.npy',allow_pickle=True)
train_words = np.load('./drive/My Drive/Thesis Work/WordLevel_RL_MP_MFCC_TrainCamb_TestL2/train_words.npy',allow_pickle=True)
y_train = np.load('./drive/My Drive/Thesis Work/WordLevel_RL_MP_MFCC_TrainCamb_TestL2/y_train.npy',allow_pickle=True)
model = load_model('./drive/My Drive/Thesis Work/WordLevel_RL_MP_MFCC_TrainCamb_TestL2/model.h5')
encoder = LabelBinarizer()
encoder.fit(train_words)
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(index=-2).output)

x_test = np.load('./drive/My Drive/Thesis Work/WordLevel_RL_MP_MFCC_TrainL2_TestL2/x_test.npy',allow_pickle=True)
test_phoneErrorPercentage = np.load('./drive/My Drive/Thesis Work/WordLevel_RL_MP_MFCC_TrainL2_TestL2/test_phoneErrorPercentage.npy',allow_pickle=True)
test_words = np.load('./drive/My Drive/Thesis Work/WordLevel_RL_MP_MFCC_TrainL2_TestL2/test_words.npy',allow_pickle=True)

TypeError: ignored

In [0]:
classes = list(train_words) #As these are all unique so list will work. Don't have to use np.argwhere here

In [0]:
len(set(test_words)) #Unique test words

262

In [0]:
len(set([word for word in test_words if word in classes])) #test words in Vocab

159

In [0]:
scrapEm = set([word for word in test_words if word not in classes])

Almost half words missing in train but I can scrap those from Cambridge.  
Should have scrapped these from Cambridge.  
If time do that - Scrap Test Words.

In [0]:
test_words.shape

(9140,)

In [0]:
train_representations = intermediate_layer_model.predict(x_train)
train_representations.shape

(3422, 256)

In [0]:
#VALIDATION
valSetLength = 1000 #initial
tolerancePercentage = 40

truelabels = [1 if test_phoneErrorPercentage[i] <= tolerancePercentage else 0 for i in range(valSetLength) if test_words[i] in train_words]

accs = []
f1mis = []
for threshold in [1150,1180,1190,1200]:
  predictedLabels = []
  for i in range(valSetLength): #iterating on test words
    if test_words[i] in train_words:
      if np.linalg.norm(train_representations[classes.index(test_words[i])] \
                        - intermediate_layer_model.predict(x_test[i:i+1])) <= threshold:
        predictedLabels.append(1)
      else: 
        predictedLabels.append(0)

  accs.append(sklm.accuracy_score(truelabels,predictedLabels))
  f1mis.append(sklm.precision_recall_fscore_support(truelabels,predictedLabels)[2][0])


print("Threshold: ", np.argmax(f1mis))
print("Max Achievable Accuracy: ", accs[np.argmax(f1mis)])
print("Max Achievable F-1 score for Mispronunciations: ", max(f1mis))

Threshold:  1
Max Achievable Accuracy:  0.8362068965517241
Max Achievable F-1 score for Mispronunciations:  0.32941176470588235


In [0]:
605/(605+91) #predicting all +ve :/ (Remove class imbalance good results)

0.8692528735632183

In [0]:
#TESTING
valSetLength = 1000 #initial
tolerancePercentage = 40
threshold = 1180

predictedLabels = []
for i in range(valSetLength,len(test_words)): #iterating on test words
  if test_words[i] in train_words:
    if np.linalg.norm(train_representations[classes.index(test_words[i])] \
                      - intermediate_layer_model.predict(x_test[i:i+1])) <= threshold:
      predictedLabels.append(1)
    else: 
      predictedLabels.append(0)

  
truelabels = [1 if test_phoneErrorPercentage[i] <= tolerancePercentage else 0 for i in range(valSetLength,len(test_words)) if test_words[i] in train_words]
sklm.accuracy_score(truelabels,predictedLabels)

0.8444172208610431

### Results

In [0]:
print('Accuracy = ', sklm.accuracy_score(truelabels,predictedLabels))

Accuracy =  0.8444172208610431


In [0]:
sklm.precision_recall_fscore_support(truelabels,predictedLabels) #0 - 1

(array([0.36258993, 0.91113768]),
 array([0.36103152, 0.91168262]),
 array([0.36180905, 0.91141006]),
 array([ 698, 5016]))

In [0]:
sklm.precision_recall_fscore_support(truelabels,predictedLabels)[2][0] #0

0.3618090452261307

In [0]:
sklm.roc_curve(truelabels,predictedLabels)

(array([0.        , 0.63896848, 1.        ]),
 array([0.        , 0.91168262, 1.        ]),
 array([2, 1, 0]))

*CANNOT COMPARE RESULTS:*  
The Interspeech one has a **different dataset** and shows results on phone level (as every phone is a separate class there)

The l2-arctic one shows no results

I am the first one to show results on this dataset